# Moving Average Crossover Strategy

Classic technical analysis strategy:
- **Buy Signal:** Fast MA crosses above slow MA (golden cross)
- **Sell Signal:** Fast MA crosses below slow MA (death cross)
- Uses 50-day and 200-day moving averages

## Setup

In [ ]:
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from zipline import run_algorithm
from zipline.api import (
    order_target_percent,
    symbol,
    record,
    get_open_orders,
)
from zipline.utils.progress import enable_progress_logging
from zipline.utils.flightlog_client import enable_flightlog

# Enable logging
logging.basicConfig(level=logging.INFO, force=True)

# Connect to FlightLog (optional)
# enable_flightlog(host='flightlog', port=9020)

# Enable progress tracking
enable_progress_logging(algo_name='MA-Crossover', update_interval=10)

## Strategy Configuration

In [ ]:
# Strategy parameters
FAST_MA = 50   # Fast moving average period
SLOW_MA = 200  # Slow moving average period
STOCK = 'SPY'  # Stock to trade

## Strategy Implementation

In [ ]:
def initialize(context):
    """
    Called once at the start of the algorithm.
    """
    context.stock = symbol(STOCK)
    
    # Track moving averages
    context.fast_ma = FAST_MA
    context.slow_ma = SLOW_MA
    
    # Need price history for MAs
    context.price_history = []
    
    # Track position
    context.invested = False
    
    # Trade counters
    context.buy_count = 0
    context.sell_count = 0
    
    logging.info(f"Strategy initialized")
    logging.info(f"  Stock: {STOCK}")
    logging.info(f"  Fast MA: {FAST_MA} days")
    logging.info(f"  Slow MA: {SLOW_MA} days")


def handle_data(context, data):
    """
    Called every trading day.
    """
    # Get current price
    price = data.current(context.stock, 'price')
    
    # Get price history for moving averages
    hist = data.history(
        context.stock,
        'close',
        context.slow_ma + 1,  # Need enough data for slow MA
        '1d'
    )
    
    # Calculate moving averages
    fast_mavg = hist[-context.fast_ma:].mean()
    slow_mavg = hist[-context.slow_ma:].mean()
    
    # Previous day MAs (for crossover detection)
    prev_fast_mavg = hist[-(context.fast_ma+1):-1].mean()
    prev_slow_mavg = hist[-(context.slow_ma+1):-1].mean()
    
    # Detect crossovers
    golden_cross = (prev_fast_mavg <= prev_slow_mavg) and (fast_mavg > slow_mavg)
    death_cross = (prev_fast_mavg >= prev_slow_mavg) and (fast_mavg < slow_mavg)
    
    # Check for open orders (don't trade if already have pending order)
    open_orders = get_open_orders()
    
    # Trading logic
    if golden_cross and not context.invested and not open_orders:
        # BUY signal - fast MA crossed above slow MA
        order_target_percent(context.stock, 1.0)
        context.invested = True
        context.buy_count += 1
        logging.info(f"BUY {STOCK} at ${price:.2f} (Fast MA: ${fast_mavg:.2f}, Slow MA: ${slow_mavg:.2f})")
    
    elif death_cross and context.invested and not open_orders:
        # SELL signal - fast MA crossed below slow MA
        order_target_percent(context.stock, 0.0)
        context.invested = False
        context.sell_count += 1
        logging.info(f"SELL {STOCK} at ${price:.2f} (Fast MA: ${fast_mavg:.2f}, Slow MA: ${slow_mavg:.2f})")
    
    # Record values for analysis
    record(
        price=price,
        fast_ma=fast_mavg,
        slow_ma=slow_mavg,
        invested=1 if context.invested else 0,
        portfolio_value=context.portfolio.portfolio_value
    )


def analyze(context, perf):
    """
    Called after the backtest completes.
    """
    logging.info("\n" + "="*60)
    logging.info("BACKTEST RESULTS")
    logging.info("="*60)
    
    # Calculate returns
    start_value = perf['portfolio_value'].iloc[0]
    end_value = perf['portfolio_value'].iloc[-1]
    total_return = (end_value - start_value) / start_value * 100
    
    logging.info(f"Start date: {perf.index[0].date()}")
    logging.info(f"End date: {perf.index[-1].date()}")
    logging.info(f"")
    logging.info(f"Starting portfolio value: ${start_value:,.2f}")
    logging.info(f"Ending portfolio value: ${end_value:,.2f}")
    logging.info(f"Total return: {total_return:.2f}%")
    logging.info(f"")
    logging.info(f"Number of buys: {context.buy_count}")
    logging.info(f"Number of sells: {context.sell_count}")
    logging.info(f"Total trades: {context.buy_count + context.sell_count}")
    logging.info("="*60)

## Run Backtest

In [ ]:
# Run the backtest
results = run_algorithm(
    start=pd.Timestamp('2018-01-01'),
    end=pd.Timestamp('2023-12-31'),
    initialize=initialize,
    handle_data=handle_data,
    analyze=analyze,
    capital_base=100000,
    data_frequency='daily',
    bundle='sharadar',
)

## Visualize Results

Plot price, moving averages, and buy/sell signals.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Plot 1: Price with MAs
axes[0].plot(results.index, results['price'], label=f'{STOCK} Price', linewidth=1.5, alpha=0.7)
axes[0].plot(results.index, results['fast_ma'], label=f'{FAST_MA}-day MA', linewidth=2)
axes[0].plot(results.index, results['slow_ma'], label=f'{SLOW_MA}-day MA', linewidth=2)

# Mark buy/sell signals
buy_signals = results[results['invested'].diff() == 1]
sell_signals = results[results['invested'].diff() == -1]

if not buy_signals.empty:
    axes[0].scatter(buy_signals.index, buy_signals['price'], 
                   marker='^', color='green', s=100, label='Buy', zorder=5)
if not sell_signals.empty:
    axes[0].scatter(sell_signals.index, sell_signals['price'], 
                   marker='v', color='red', s=100, label='Sell', zorder=5)

axes[0].set_ylabel('Price ($)', fontsize=12)
axes[0].set_title(f'Moving Average Crossover Strategy: {STOCK}', fontsize=14, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Plot 2: Portfolio value
axes[1].plot(results.index, results['portfolio_value'], linewidth=2, color='purple')
axes[1].set_ylabel('Portfolio Value ($)', fontsize=12)
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
axes[1].grid(True, alpha=0.3)

# Plot 3: Position indicator
axes[2].fill_between(results.index, 0, results['invested'], 
                     where=results['invested']==1, 
                     color='green', alpha=0.3, label='In position')
axes[2].set_ylabel('Position', fontsize=12)
axes[2].set_xlabel('Date', fontsize=12)
axes[2].set_yticks([0, 1])
axes[2].set_yticklabels(['Cash', 'Invested'])
axes[2].legend(loc='best')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Performance Metrics

In [ ]:
# Calculate performance metrics
total_return = ((results['portfolio_value'].iloc[-1] / results['portfolio_value'].iloc[0]) - 1) * 100
daily_returns = results['portfolio_value'].pct_change().dropna()
sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252)  # Annualized
max_drawdown = ((results['portfolio_value'] / results['portfolio_value'].cummax()) - 1).min() * 100

# Count trades
n_buys = len(results[results['invested'].diff() == 1])
n_sells = len(results[results['invested'].diff() == -1])

print("\n" + "="*60)
print("PERFORMANCE SUMMARY")
print("="*60)
print(f"Total Return: {total_return:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Max Drawdown: {max_drawdown:.2f}%")
print(f"")
print(f"Final Portfolio Value: ${results['portfolio_value'].iloc[-1]:,.2f}")
print(f"Number of Buy Signals: {n_buys}")
print(f"Number of Sell Signals: {n_sells}")
print(f"Total Trades: {n_buys + n_sells}")
print("="*60)

## Trade Log

In [ ]:
# Extract all trades
trades = []
for date, row in results.iterrows():
    if len(row['transactions']) > 0:
        for txn in row['transactions']:
            trades.append({
                'Date': date.date(),
                'Action': 'BUY' if txn['amount'] > 0 else 'SELL',
                'Shares': abs(txn['amount']),
                'Price': txn['price'],
                'Value': abs(txn['amount'] * txn['price']),
                'Portfolio Value': row['portfolio_value']
            })

if trades:
    trades_df = pd.DataFrame(trades)
    print("\nAll Trades:")
    print(trades_df.to_string(index=False))
else:
    print("\nNo trades executed.")

## Next Steps

**Experiment with parameters:**
- Try different MA periods (e.g., 20/50, 10/30)
- Test on different stocks
- Add stop-loss or take-profit rules
- Combine with other indicators (RSI, MACD)

**Further analysis:**
- See `04_pyfolio_analysis.ipynb` for detailed risk/return metrics
- Compare with buy-and-hold strategy
- Test on different time periods